In [ ]:
import os
import sys

import mlflow
from mlflow.client import MlflowClient
from mlflow.models import infer_signature, ModelSignature
from mlflow.types import Schema, ColSpec

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import datasets
import pandas as pd

### Model Training Phase

In [ ]:
# Loading data
data = datasets.load_breast_cancer()
# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data.data, 
                                                    data.target,
                                                    stratify=data.target)
# Instantiating and fitting the model
model = LogisticRegression(max_iter=1000)            
model.fit(X=X_train, y=y_train)


In [ ]:
# Converting train features into a DataFrame
X_train_df = pd.DataFrame(data=X_train, columns=data.feature_names)

# Inferring the input signature
signature = infer_signature(model_input=X_train_df, 
                           model_output=model.predict(X_test))
signature

In [ ]:
# Creating an input schema for the breast cancer dataset
input_schema = Schema(inputs=[ColSpec(type="double", name=feature_name) 
                              for feature_name in data.feature_names])

# Creating an output schema for the breast cancer dataset
output_schema = Schema(inputs=[ColSpec("double")])

#Creating a signature from our schemas
signature_manual = ModelSignature(inputs=input_schema, outputs=output_schema)
signature_manual

### Save the model locally to /tmp/mymodel

This is just to show how you can save and run the model locally

In [ ]:
import os
import shutil
folder_path = "/tmp/mymodel"
if os.path.exists(folder_path):
        if os.path.isdir(folder_path):
            shutil.rmtree(folder_path)
            
# Saving the model. Note the path. This will save the model under /mnt/model
input_example = X_train_df.iloc[:1]
mlflow.sklearn.save_model(sk_model=model, 
                          path=folder_path, 
                          signature=signature,
                          input_example=input_example)
##Verify that is looks good
os.listdir(folder_path)

### Review the output

Especially take a look at the `requirements.txt` and the yaml files and the `requirements.txt`

### Run the locally saved model

This is a way all models should be run if you want them to be portable. This is the industry
standard MLFLOW based mechanism to load and run models.

In [ ]:
#Run the locally saved model
import os
import pandas as pd
def predict(model_uri,features):
    loaded_model = mlflow.pyfunc.load_model(model_uri)
    return loaded_model.predict(features)
cwd = os.getcwd()
print(cwd)



In [ ]:
d = pd.read_json(f'{cwd}/client/features.json', orient='records', lines=True)    
predict(folder_path,d)

### Now we register this model with Domino Experiment Manager

1. Create an experiment with a meaningful name
2. Create a registered model name
3. Finally register model

In [ ]:
prefix = 'TUTORIAL-COMPLEX-LOGGGING'
starting_user_name = os.environ['DOMINO_STARTING_USERNAME']
project_name = os.environ['DOMINO_PROJECT_NAME']

experiment_name = f'{prefix}-{starting_user_name}-{project_name}'
model_name = f'model-{prefix}-{starting_user_name}-{project_name}'


client = MlflowClient()
try:
    client.create_registered_model(model_name)
except:
    print('Model already exists')

## The Most Important Part - Model Registration

Pay close attention to not just the models that are being registered. But also the additional files we are choosing to add to the model registry. We can add anything our final image in our 
final execution environment will need

In [ ]:
# Saving the model as an artifact in a run
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository
mlflow.set_experiment(experiment_name)


run_id=''
##Specify Dependencies implicitly
with mlflow.start_run() as run:
    # Obtaining the ID of this run
    run_id = run.info.run_id
    # Logging our model
    #model_folder = 'mymodel'
    model_folder = ''
    model_client_folder = 'client'
    model_templates_folder = 'templates'
    model_entry_point = 'python'
    model_command_line = 'client/execute_model.py'
    model_info = mlflow.sklearn.log_model(sk_model=model, 
                             artifact_path=model_folder,  
                             signature=signature,
                             input_example=input_example)
    ##Note these artifacts being logged.
    mlflow.log_artifact(f'{cwd}/client/model.json',model_client_folder)
    mlflow.log_artifact(f'{cwd}/client/features.json',model_client_folder)
    mlflow.log_artifact(f'{cwd}/client/example_predict.py',model_client_folder)
    mlflow.log_artifact(f'{cwd}/client/execute_model.py',model_client_folder)
    mlflow.log_artifact(f'{cwd}/templates/Dockerfile.template',model_templates_folder)
    mlflow.log_artifact(f'{cwd}/templates/create_docker_image.sh.template',model_templates_folder)

'''
Tags are a way of passing metadata to the model version client. In our case it will be the 
external program that will download these model versions and publish images to foundry

'''
my_tags={}
my_tags['MODEL_FOLDER']='mymodel'
my_tags['MODEL_CLIENT_FOLDER']='client'
my_tags['MODEL_ENTRY_POINT']='python'
my_tags['MODEL_EXECUTE_PATH']='client/execute_model.py'

model_src = RunsArtifactRepository.get_underlying_uri(f"runs:/{run_id}/")
mv = client.create_model_version(model_name, model_src, run_id,tags=my_tags)
print("Name: {}".format(mv.name))
print("Version: {}".format(mv.version))
print("Description: {}".format(mv.description))
print("Status: {}".format(mv.status))
print("Stage: {}".format(mv.current_stage))

In [ ]:
## Now fetch the model from model registry

In [ ]:
model_version=4
mv = client.get_model_version(model_name, model_version)
print('Model Version')
print(mv)

run_id = mv.run_id
print('Model Run Id')
print(run_id)
model_download_path = f'/tmp/models/{model_name}/v{model_version}'
os.makedirs(model_download_path, exist_ok=True)
print(f'Model Download Path {model_download_path}')
client.download_artifacts(run_id, f"", model_download_path)

In [ ]:
!ls /tmp/models/model-TUTORIAL-COMPLEX-LOGGGING-integration-test-Experiment-Manager-Demos/v1/

In [ ]:
import string
def create_file_from_template(template_string, context, output_file_path):
    template = string.Template(template_string)
    content = template.safe_substitute(context)
    with open(output_file_path, 'w') as file:
        file.write(content)

In [ ]:
templates_folder = 'templates'
context = {
        'model_name': model_name,
        'model_version': model_version,
        'model_download_folder': model_download_path,
        'model_folder': mv.tags['MODEL_FOLDER'],
        'model_client_folder': mv.tags['MODEL_CLIENT_FOLDER'],
        'entry_point': mv.tags['MODEL_ENTRY_POINT'],
        'command_line': mv.tags['MODEL_EXECUTE_PATH'],
}

if 'MODEL_TEMPLATES_FOLDER' in context:
    templates_folder = context['MODEL_TEMPLATES_FOLDER']
with open(f"{model_download_path}/{templates_folder}/Dockerfile.template", 'r') as file:
    content = file.read()
    output_file = f"{model_download_path}/Dockerfile"
    create_file_from_template(content, context, output_file)
with open(f"{model_download_path}/{templates_folder}/create_docker_image.sh.template", 'r') as file:
    content = file.read()
    output_file = f"{model_download_path}/create_docker_image.sh"    
    create_file_from_template(content, context, output_file)

In [ ]:
#Re Run the locally retrieved model
print(model_download_path)
cwd = os.getcwd()
d = pd.read_json(f'{cwd}/client/features.json', orient='records', lines=True)    

In [ ]:
#Re Run the locally retrieved model
import os
import pandas as pd
def predict(model_uri,features):
    loaded_model = mlflow.pyfunc.load_model(model_uri)
    return loaded_model.predict(features)
model_path = f"{model_download_path}"
d = pd.read_json(f'{model_download_path}/client/features.json', orient='records', lines=True)
result = predict(model_path,d)

print("Model Result")
print(result)

print("Input Dataset")
print(d)
        